In [15]:
# -----------------------------
# ONLINE PHASE - IMPLEMENTACIÓN
# -----------------------------

from sage.all import GF
from hashlib import sha256

##########################
# Clase AuthShare mínima #
##########################
class AuthShare:
    """
    Representa una compartición autenticada <x>:
      - F: campo (Sage GF)
      - delta: posible offset (usado por conveniencia)
      - shares: lista de shares (cada jugador tiene su share)
      - macs: lista de mac-shares (sum(macs) = alpha * (value + delta))
    Métodos:
      - value(): reconstruye el valor secreto x (suma de shares)
      - mac_sum(): suma de mac-shares
      - n_players(): número de jugadores
    """
    def __init__(self, F, delta, shares, macs):
        self.F = F
        self.delta = F(delta)
        self.shares = [F(s) for s in shares]
        self.macs = [F(m) for m in macs]

    def value(self):
        return sum(self.shares)

    def mac_sum(self):
        return sum(self.macs)

    def n_players(self):
        return len(self.shares)

##########################
# Helper: generar AuthShare de prueba (preprocessing simulado)
##########################
def make_rand_auth(F, v, alpha, n):
    """
    Crea una AuthShare simulada:
      - shares aleatorias que suman v
      - macs aleatorias que suman alpha*(v + delta)
    Útil para pruebas unitarias y demos.
    """
    shares = [F.random_element() for _ in range(n-1)]
    shares.append(F(v - sum(shares)))
    delta = F(0)
    mac_target = alpha * (v + delta)
    macs = [F.random_element() for _ in range(n-1)]
    macs.append(mac_target - sum(macs))
    return AuthShare(F, delta, shares, macs)

####################################################
# Modo A: convención práctica (ajuste en la última mac)
####################################################
def input_phase(F, players, x_input, R_share, alpha):
    """
    Input protocol (modo A: convención 'ajustar la última mac-share'):
      - R_share: AuthShare de un mask <r> (preprocessing)
      - x_input: valor a introducir (elemento de F)
      - alpha: clave global (elemento de F)
    Retorna: (AuthShare de <x>, eps abierto público)
    """
    # 1) reconstruir r
    r_val = R_share.value()

    # 2) epsilon público
    eps = F(x_input - r_val)

    # 3) nuevas shares: r_i + eps
    new_shares = [F(s + eps) for s in R_share.shares]

    # 4) ajuste macs: sum(new_macs) = alpha*(x + delta)
    new_macs = [F(m) for m in R_share.macs]
    new_macs[-1] = new_macs[-1] + (eps * alpha)

    return AuthShare(F, R_share.delta, new_shares, new_macs), eps

def multiply_phase(F, X, Y, triple, alpha):
    """
    Multiplicación usando triple A,B,C (modo A).
    - X,Y: AuthShare operandos
    - triple: (A,B,C) AuthShare con A*B = C
    - alpha: clave global
    Devuelve: (Z_auth, eps, delta)
    """
    A, B, C = triple
    n = A.n_players()
    assert n == B.n_players() == C.n_players() == X.n_players() == Y.n_players()

    # Abrir e = X - A , d = Y - B (público)
    e_val = X.value() - A.value()
    d_val = Y.value() - B.value()
    eps = F(e_val)
    delta = F(d_val)

    # Z_i = C_i + eps * B_i + delta * A_i (añadir eps*delta una sola vez)
    Z_shares = [
        F(C.shares[i] + eps * B.shares[i] + delta * A.shares[i])
        for i in range(n)
    ]
    # Añadir eps*delta solo una vez (en la última share por convención)
    Z_shares[-1] = Z_shares[-1] + (eps * delta)

    # MACs: propagar y añadir eps*delta*alpha en la última mac-share
    Z_macs = [
        F(C.macs[i] + eps * B.macs[i] + delta * A.macs[i])
        for i in range(n)
    ]
    Z_macs[-1] = Z_macs[-1] + (eps * delta * alpha)

    return AuthShare(F, C.delta, Z_shares, Z_macs), eps, delta

def output_phase(F, players, opened_so_far, Z, alpha):
    """
    Output / MAC check (modo A).
    Verifica: sum(macs) == alpha * sum(shares)
    Retorna: (ok_bool, valor_abierto_z)
    """
    z_val = Z.value()
    mac_val = Z.mac_sum()
    ok = (mac_val == alpha * z_val)
    return ok, z_val

def reshare(F, players, X, R_share_for_reshare, alpha):
    """
    Reshare: refrescar compartición <x> usando mask R_share_for_reshare.
    - R_share_for_reshare: AuthShare de un random r
    Retorna: nuevo AuthShare de <x>
    """
    r_val = R_share_for_reshare.value()
    eps = F(X.value() - r_val)
    new_shares = [F(s + eps) for s in R_share_for_reshare.shares]
    new_macs = [F(m) for m in R_share_for_reshare.macs]
    new_macs[-1] = new_macs[-1] + (eps * alpha)
    return AuthShare(F, R_share_for_reshare.delta, new_shares, new_macs)

####################################################
# Modo B: soporte para alpha_shares distribuidas
####################################################
def input_phase_distributed(F, players, x_input, R_share, alpha_shares):
    """
    Input Phase versión distribuida: se reparte el ajuste de macs
    según alpha_shares (lista de partes de alpha por jugador).
    alpha_shares debe ser lista de la misma longitud que R_share.shares.
    """
    r_val = R_share.value()
    eps = F(x_input - r_val)
    new_shares = [F(s + eps) for s in R_share.shares]

    # Ajuste distribuido: añadir eps * alpha_shares[i] a cada mac_share[i]
    new_macs = [F(m) for m in R_share.macs]
    for i in range(len(new_macs)):
        new_macs[i] = new_macs[i] + (eps * F(alpha_shares[i]))

    return AuthShare(F, R_share.delta, new_shares, new_macs), eps

def multiply_phase_distributed(F, X, Y, triple, alpha_shares):
    """
    Multiply Phase versión distribuida:
    - alpha_shares: lista de porciones de alpha para cada jugador
    """
    A, B, C = triple
    n = A.n_players()
    assert len(alpha_shares) == n

    e_val = X.value() - A.value()
    d_val = Y.value() - B.value()
    eps = F(e_val)
    delta = F(d_val)

    Z_shares = [
        F(C.shares[i] + eps * B.shares[i] + delta * A.shares[i])
        for i in range(n)
    ]
    Z_shares[-1] = Z_shares[-1] + (eps * delta)  # eps*delta solo una vez

    Z_macs = [
        F(C.macs[i] + eps * B.macs[i] + delta * A.macs[i])
        for i in range(n)
    ]
    # distribuir eps*delta*alpha por porciones
    for i in range(n):
        Z_macs[i] = Z_macs[i] + (eps * delta * F(alpha_shares[i]))

    return AuthShare(F, C.delta, Z_shares, Z_macs), eps, delta

def reshare_distributed(F, players, X, R_share_for_reshare, alpha_shares):
    r_val = R_share_for_reshare.value()
    eps = F(X.value() - r_val)
    new_shares = [F(s + eps) for s in R_share_for_reshare.shares]
    new_macs = [F(m) for m in R_share_for_reshare.macs]
    for i in range(len(new_macs)):
        new_macs[i] = new_macs[i] + (eps * F(alpha_shares[i]))
    return AuthShare(F, R_share_for_reshare.delta, new_shares, new_macs)

##########################
# Demo completo (sanity)
##########################
def demo_online(mode='A'):
    """
    Ejecuta un flujo completo online (input, multiply, output) y muestra checks.
    mode: 'A' usa convención práctica (última mac-share).
          'B' usa alpha_shares distribuidas (requieres alpha_shares generadas).
    """
    p = 2**61 - 1
    F = GF(p)
    n = 3

    # alpha global (si usar modo B, vamos a repartirlo)
    alpha = F.random_element()

    print("=== Demo Online (modo {}) ===".format(mode))
    print("Campo p =", p, ", jugadores n =", n)
    print("alpha (global) =", int(alpha))

    # Simular preprocessing: dos masks R1,R2 y triple A,B,C
    R1 = make_rand_auth(F, F.random_element(), alpha, n)
    R2 = make_rand_auth(F, F.random_element(), alpha, n)
    a = F.random_element(); b = F.random_element(); c = a*b
    A = make_rand_auth(F, a, alpha, n)
    B = make_rand_auth(F, b, alpha, n)
    C = make_rand_auth(F, c, alpha, n)
    triple = (A,B,C)

    # Inputs del participante
    x_plain = F(1234)
    y_plain = F(55)

    if mode == 'A':
        X, eps1 = input_phase(F, None, x_plain, R1, alpha)
        Y, eps2 = input_phase(F, None, y_plain, R2, alpha)
    else:
        # repartir alpha en partes aleatorias que sumen alpha
        alpha_shares = [F.random_element() for _ in range(n-1)]
        alpha_shares.append(alpha - sum(alpha_shares))
        print("alpha_shares:", [int(a) for a in alpha_shares])
        X, eps1 = input_phase_distributed(F, None, x_plain, R1, alpha_shares)
        Y, eps2 = input_phase_distributed(F, None, y_plain, R2, alpha_shares)

    # Multiply
    if mode == 'A':
        Z, e, d = multiply_phase(F, X, Y, triple, alpha)
    else:
        Z, e, d = multiply_phase_distributed(F, X, Y, triple, alpha_shares)

    # OUTPUT check
    if mode == 'A':
        ok, result = output_phase(F, None, [X, Y], Z, alpha)
    else:
        # en modo distribuido la verificación global iguala sum(macs) ?= alpha * z
        ok = (Z.mac_sum() == alpha * Z.value())
        result = Z.value()

    print("eps:", int(eps1), int(eps2))
    print("z_val (reconstruido):", int(result))
    print("mac_sum:", int(Z.mac_sum()))
    print("alpha * z:", int(alpha * result))
    print("MAC correctos?:", ok)
    print("Resultado esperado (x*y mod p):", int((x_plain * y_plain) % p))

    return ok

# Ejecuta demo rápido en modo A (convención)
okA = demo_online(mode='A')
# Ejecuta demo en modo B (distribuido)
okB = demo_online(mode='B')


=== Demo Online (modo A) ===
Campo p = 2305843009213693951 , jugadores n = 3
alpha (global) = 1955461340645985684
eps: 975523761207882666 793083106229731587
z_val (reconstruido): 1694057164030991697
mac_sum: 2103187742900935140
alpha * z: 2103187742900935140
MAC correctos?: True
Resultado esperado (x*y mod p): 67870
=== Demo Online (modo B) ===
Campo p = 2305843009213693951 , jugadores n = 3
alpha (global) = 2089644755023812360
alpha_shares: [679938559228572668, 24217976468391093, 1385488219326848599]
eps: 1014163325907636542 1811895170409133836
z_val (reconstruido): 1103763532146020110
mac_sum: 1274634796849544743
alpha * z: 1274634796849544743
MAC correctos?: True
Resultado esperado (x*y mod p): 67870
